In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Set batch size
batch_size = 2048

# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Initialize the CNN model
model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
epochs = 300
init_start_time = time.time()

for epoch in range(epochs):
    start_time = time.time()
    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch: {epoch+1}, Loss: {running_loss/len(trainloader)}, Training Time: {training_time:.2f} seconds')

    total_end_time = time.time()
    total_train_time = total_end_time - init_start_time
    print(f'Total Training Time: {total_train_time:.2f} seconds')

    # Validation accuracy
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {100*accuracy:.2f}%')

    model.train()

print(f'Final Test Accuracy: {100*accuracy:.2f}%')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch: 1, Loss: 2.302675313949585, Training Time: 9.96 seconds
Total Training Time: 9.96 seconds
Test Accuracy: 12.67%
Total Training Time: 24.05 seconds
Test Accuracy: 12.90%
Total Training Time: 37.99 seconds
Test Accuracy: 12.95%
Total Training Time: 52.05 seconds
Test Accuracy: 12.83%
Total Training Time: 65.99 seconds
Test Accuracy: 12.78%
Total Training Time: 79.90 seconds
Test Accuracy: 12.73%
Total Training Time: 93.87 seconds
Test Accuracy: 12.77%
Total Training Time: 107.98 seconds
Test Accuracy: 12.81%
Total Training Time: 121.92 seconds
Test Accuracy: 12.93%
Total Training Time: 135.86 seconds
Test Accuracy: 13.15%
Epoch: 11, Loss: 2.2906062698364256, Training Time: 9.10 seconds
Total Training Time: 149.85 seconds
Test Accuracy: 13.43%
Total Training Time: 163.80 seconds
Test Accuracy: 13.72%
Total Training Time: 177.79 seconds
Test Accuracy: 14.06%
Total Traini